In [1]:
#source https://github.com/pytorch/examples/blob/master/mnist/main.py

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
class Argument():
    def __init__(self, batch_size=64, test_batch_size=1000,epochs=14, lr=1.0,
                gamma=0.7,no_cuda=False, log_interval=100,save_model=False):
        
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size
        self.epochs = epochs
        self.lr = lr
        self.gamma = gamma
        self.no_cuda = no_cuda
        self.log_interval = log_interval
        self.save_model = save_model


In [3]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


def main():
    args = Argument()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


In [4]:
%%timeit -n 1 -r 2
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299212
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.277904
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.316506
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.176994
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.245506
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.058678
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.096332
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.166872
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.107573
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.174944

Test set: Average loss: 0.0507, Accuracy: 9838/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.056642
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.032684
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.093746
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.010138
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.015474
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.100619
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.176754
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.041985
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.010226
T

Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.086300
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.057260

Test set: Average loss: 0.0466, Accuracy: 9855/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.022379
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.091989
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.074851
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.012726
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.219235
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.039863
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.045708
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.092280
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.051113
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.019846

Test set: Average loss: 0.0392, Accuracy: 9869/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.050926
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.044986
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.001592
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.019539
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.037825
Train Epoch: 3 [32000/60000 (53%)]	Loss: